In [2]:
import pandas as pd 
import sys
import os
sys.path.append(os.path.abspath(r'C:\Users\Yibabe\Desktop\kifiyaAIMweek-2\scripts'))
from task_4_connection import load_data_from_postgres

In [3]:
# Define the query to fetch data from the 'cleaned_data' table
query = "SELECT * FROM cleaned_data"

# Load the data into a DataFrame
df_cleaned = load_data_from_postgres(query)

# Display the first few rows of the DataFrame to verify
df_cleaned.head()

C:\Users\Yibabe\Desktop\kifiyaAIMweek-2\scripts\task_4_connection.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,Dur. (ms),Start,End,IMSI,MSISDN/Number,Email DL (Bytes),Gaming DL (Bytes),Google DL (Bytes),Avg RTT DL (ms),Avg Bearer TP DL (kbps),...,HTTP DL (Bytes),HTTP UL (Bytes),Total DL (Bytes),Total UL (Bytes),Social Media DL (Bytes),Youtube DL (Bytes),Netflix DL (Bytes),Handset Type,Handset Manufacturer,Bearer Id
0,951292.0,4/14/2019 2:11,4/25/2019 2:26,2.082010e+14,33763490140.0,2987408.0,748906189.0,1304524.0,39.0,3698.0,...,230583.0,25427.0,775350343.0,58813016.0,3103940.0,12189786.0,6858496.0,undefined,undefined,1.304243e+19
1,941634.0,4/14/2019 12:48,4/25/2019 10:22,2.082014e+14,33698743617.0,3014208.0,801543830.5,7560820.0,45.0,46.0,...,0.0,0.0,861612261.0,42363146.0,342043.0,18436641.0,10587264.0,Samsung Galaxy A8 (2018),Samsung,1.311448e+19
2,864482.0,4/15/2019 0:32,4/25/2019 0:40,2.082003e+14,33659219748.0,522457.0,801543830.5,8601949.0,97.0,3845.0,...,132246.0,32449.0,850570347.0,22417975.0,1541259.0,16338345.0,12211149.0,Samsung Galaxy Note 8 (Sm-N950F Ds),Samsung,1.304243e+19
3,848637.0,4/15/2019 10:30,4/25/2019 6:14,2.082098e+14,33665646348.0,3095404.0,565283082.0,4366381.0,45.0,6.0,...,0.0,0.0,594967693.0,56559490.0,2653333.0,2595678.0,16973815.0,undefined,undefined,7.277826e+18
4,889834.0,4/15/2019 11:33,4/25/2019 18:44,2.082019e+14,33664473872.0,1541381.0,504604946.0,7241855.0,217.0,28305.0,...,35986300.0,689004.0,535118044.0,33691738.0,829305.0,2805710.0,18094847.0,Huawei B593S-22,Huawei,1.311448e+19


In [5]:
import numpy as np
import pandas as pd

# Assuming df_cleaned is your DataFrame with user data
# These should be the actual columns related to engagement and experience
engagement_columns = ['Total DL (Bytes)', 'Total UL (Bytes)']
experience_columns = ['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']

# Assuming cluster_center_engagement and cluster_center_experience represent the centers of the less engaged and worst experience clusters
cluster_center_engagement = np.array([500000, 200000])  # Example center for engagement
cluster_center_experience = np.array([100, 50])         # Example center for experience

# Define a function to calculate Euclidean distance
def euclidean_distance(user_data, cluster_center):
    return np.sqrt(np.sum((user_data - cluster_center) ** 2))

# Add engagement and experience scores
df_cleaned['engagement_score'] = df_cleaned.apply(
    lambda row: euclidean_distance(row[engagement_columns].values, cluster_center_engagement), axis=1
)
df_cleaned['experience_score'] = df_cleaned.apply(
    lambda row: euclidean_distance(row[experience_columns].values, cluster_center_experience), axis=1
)

# Display the DataFrame with the new scores
print(df_cleaned[['engagement_score', 'experience_score']])


        engagement_score  experience_score
0           7.770641e+08       3648.509970
1           8.621439e+08         55.145263
2           8.503606e+08       3795.001186
3           5.971333e+08         70.434367
4           5.356661e+08      28255.242239
...                  ...               ...
148987      8.202660e+08      63370.113618
148988      5.765426e+08         68.029405
148989      6.672857e+08         77.833155
148990      5.933027e+08         57.428216
148991      3.725598e+08         65.000000

[148992 rows x 2 columns]


In [7]:
# Calculate satisfaction score as average of engagement and experience scores
df_cleaned['satisfaction_score'] = (df_cleaned['engagement_score'] + df_cleaned['experience_score']) / 2

# Get top 10 satisfied users
top_10_satisfied = df_cleaned.nlargest(10, 'satisfaction_score')

# Use 'MSISDN/Number' as the user identifier
print(top_10_satisfied[['MSISDN/Number', 'satisfaction_score']])


        MSISDN/Number  satisfaction_score
60868   33781299820.0        4.515527e+08
39188   33658138959.0        4.498754e+08
69632   33665560117.0        4.498615e+08
146935  33668533742.0        4.494621e+08
37756   33661689404.0        4.487572e+08
3114    33699341099.0        4.486384e+08
76290   33658809959.0        4.483996e+08
13710   33666188130.0        4.483059e+08
66284   33668551754.0        4.479602e+08
137109  33664278721.0        4.477900e+08


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Prepare features and target
X = df_cleaned[['engagement_score', 'experience_score']]
y = df_cleaned['satisfaction_score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 3.8300569416639945e-14
R-squared: 1.0


In [9]:
from sklearn.cluster import KMeans

# Define KMeans model
kmeans = KMeans(n_clusters=2)

# Fit on engagement and experience scores
df_cleaned['cluster'] = kmeans.fit_predict(df_cleaned[['engagement_score', 'experience_score']])

# Show the cluster centers
print(kmeans.cluster_centers_)


[[6.66516816e+08 1.32542373e+04]
 [2.47615451e+08 1.33500058e+04]]


In [10]:
# Aggregate average satisfaction and experience scores by cluster
cluster_summary = df_cleaned.groupby('cluster').agg({
    'satisfaction_score': 'mean',
    'experience_score': 'mean'
}).reset_index()

print(cluster_summary)


   cluster  satisfaction_score  experience_score
0        0        3.334325e+08      13251.833245
1        1        1.239815e+08      13352.260245
